In [ ]:
!rm -r /content/sample_data

In [ ]:
%mkdir an_data

In [ ]:
%cd /content/an_data

In [ ]:
from google.colab import files #Upload the annotated text files from local drive
files.upload()

In [ ]:
import nltk, re, string, unicodedata, inflect, glob, os
import re
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import numpy as np 
import pandas as pd                                 #for data manipulation and analysis
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
for dirname, _, filenames in os.walk('/content/an_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.concat([pd.read_csv(f, sep = '\t', header = None) for f in glob.glob('*.tsv')], ignore_index = True)#bind the tsv files together in a single file

In [ ]:
df.columns = ['sentence','label']
new_gene = df.loc[df["label"] == "gene"]
new_enzyme = df.loc[df["label"] == "enzyme"]

In [ ]:
len(new_enzyme)

33

In [ ]:
new_gene.to_csv(r'/content/new_gene.csv', index = False)


In [ ]:
new_enzyme.to_csv(r'/content/new_enzyme.csv', index = False)

In [ ]:
new_gene = df.loc[df["label"] == "gene"]


In [ ]:
len(new_gene)

129

In [ ]:
%cd /content

In [ ]:
with open('manual_sent.txt', 'w') as f:
    for text in df["sentence"].tolist():
        f.write(text + '\n')

In [ ]:
with open("/content/manual_sent.txt", 'rb') as corpus:
  text_full = corpus.read()

In [ ]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text

In [ ]:
sample = denoise_text(str(text_full))

In [ ]:
erst = sent_tokenize(str(sample))

In [ ]:
erst

Pumed ids

In [ ]:
!pip install biopython
!pip install --upgrade biopython
!pip install metapub

In [ ]:
import Bio
import metapub
from metapub import PubMedFetcher
from Bio import Entrez
from Bio import SeqIO
from Bio.Entrez import efetch

In [ ]:
pmid_new = "15240330,21418495,20406281,22991035,21531434,24668245,3107671,8226980,16391099,24123738,24676750,22854895,30647456,28363960,24239819,24898178,15883866,17117994,22247165,23186529,20305027,22938719,24721596,20401686,20088886"

In [ ]:
Entrez.email = "a.sakapetis@students.uu.nl"
handle = Entrez.efetch(db="pubmed", id= pmid_new ,rettype="xml", retmode="text")
records = Entrez.read(handle)

In [ ]:
abstract_dict = {}
without_abstract = []

for pubmed_article in records['PubmedArticle']:
    pmid = int(str(pubmed_article['MedlineCitation']['PMID']))
    article = pubmed_article['MedlineCitation']['Article']
    if 'Abstract' in article:
        abstract = article['Abstract']['AbstractText'][0]
        abstract_dict[pmid] = abstract
    else:
       without_abstract.append(pmid)

print(abstract_dict)
print(without_abstract)

In [ ]:
ab = pd.DataFrame(abstract_dict.items())
ab.columns = ("Pubmid", "Abstracts")

In [ ]:
with open('abstracts_rest.txt', 'w') as f:
    for text in ab["Abstracts"].tolist():
        f.write(text)

In [ ]:
with open("/content/abstracts_rest.txt", 'rb') as corpus:
  text_full = corpus.read()

In [ ]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text

In [ ]:
sample = denoise_text(str(text_full))

In [ ]:
sec = sent_tokenize(str(sample))

In [ ]:
sec

Cohen Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score


In [ ]:
labelA = ["a", "b", "c"]
labelB = ["d", "b", "c"]

In [ ]:
biologist = pd.read_csv (r'/content/manuall annotated sentence.csv')

In [ ]:
script = pd.read_csv(r'/content/automaticall.csv')

In [ ]:
labelA = biologist['Entity'].tolist()

In [ ]:
labelB = script['Tag'].tolist()

In [ ]:
from nltk.metrics import ConfusionMatrix

In [ ]:
from nltk.metrics import *

In [ ]:
cm = ConfusionMatrix(labelA,labelB)

In [ ]:
print(cm)

       |         B         I      |
       |    B    -    I    -      |
       |    -    G    -    G      |
       |    E    E    E    E      |
       |    N    N    N    N      |
       |    Z    E    Z    E    O |
-------+--------------------------+
 B-ENZ |  <18>   .    .    .    1 |
B-GENE |    .  <86>   .    .    9 |
 I-ENZ |    .    .  <12>   .    2 |
I-GENE |    .    .    .   <.>   2 |
     O |   16   17    9    .<6488>|
-------+--------------------------+
(row = reference; col = test)



In [ ]:
import nltk


In [ ]:
cohen_kappa_score(labelA, labelB)


0.8023335621139327